# Keyframing with Vapor

In this notebook, we will create an animation where our camera moves to view our visualization from different angles. We will achieve this using an animation technique called keyframing. This technique involves selecting several key scenes, known as keyframes, and then having the computer generate the intermediate frames between these keyframes. This process creates a smooth transition from one keyframe to the next, resulting in a fluid animation.

### Setup and data download

In [34]:
from vapor import session, dataset, renderer
from vapor.animation import Animation

The following cell will download the data from NCAR's [Research Data Archives](https://rda.ucar.edu/datasets/ds897.7/dataaccess/#).

In [35]:
!if [ ! -f "./data/wrfout_d02_2005-08-29_02.nc" ]; then \
  curl -o Katrina.zip "https://data.rda.ucar.edu/ds897.7/Katrina.zip"; \
fi
!if [ -f "Katrina.zip" ]; then unzip -o Katrina.zip -d ./data; \
    rm Katrina.zip; \
fi

### Create keyframes

To begin creating our animation, we first create our keyframes. Each keyframe corresponds to a specific moment in our animation, defined by the camera's position and orientation. We can create these keyframes using either Python or the Vapor application, and save them as session files. For the purpose of this notebook, we'll input our camera information directly in Python, but in practice, it's easier to fine-tune the camera settings within the application.

First we'll create our static visualization. A detailed guide on how this is created can be found in the quickstart notebook

In [36]:
ses = session.Session()
data = ses.OpenDataset(dataset.WRF, ["data/wrfout_d02_2005-08-29_02.nc"])
land = data.NewRenderer(renderer.ImageRenderer)
# # Uncomment for BigBlueMarble with oceans
# land.SetImagePath("/private/var/folders/j4/hq3hhpyd0_b0y1mcsh3_tttw4k2wf2/T/AppTranslocation/DD9F0ECB-CB47-41CE-940C-0EB8E70392DD/d/vapor.app/Contents/share/images/BigBlueMarble.tms")
# land.SetIgnoreTransparency(True)
clouds = data.NewRenderer(renderer.VolumeRenderer) # Render clouds
clouds.SetVariableName("QCLOUD")
clouds_tf = clouds.GetTransferFunction("QCLOUD")
clouds_tf.LoadBuiltinColormap("Sequential/BlackWhite")
clouds_tf.SetColorRGBList([(r, g, b) for r, g, b, _ in 
                           list(reversed(clouds_tf.GetMatPlotLibColormap().colors))])
clouds_tf.SetOpacityControlPoints([[0,0],[0.00001,0.01], [0.0001, 0.1], [0.0010,0.9]])

Now, we'll define the camera settings for each of our keyframes. These numbers are tricky to get right in Python, so it is recommended that they are selected within the application.

In [37]:
positions = [
    [-1190444.44426004, 1882360.85954653, 770176.40842364], # Keyframe 1
    [-1172384.15238047, 2813172.26639064, 355291.41877028], # Keyframe 2
    [-968784.32993129, 3056725.58106798, -34317.16158186], # ...
    [-733144.08018801, 2929790.21696698, -32984.22588893],
    [-691781.20449513, 2442083.68616993, -47289.68751812]
]

targets = [
    [-420811.28125, 2737271.75, 5699.78515597],
    [-420811.28125, 2737271.75, 15699.78515597],
    [-420811.28125, 2737271.75, 15699.78515597],
    [-420811.28125, 2737271.75, 15699.78515597],
    [-420811.28125, 2737271.75, 15699.78515597]
]

ups = [
    [0.41853764, 0.35630071, 0.83538976],
    [ 0.39861183, -0.08972356, 0.91272027],
    [-0.08058301, 0.02890014, 0.99632884],
    [-0.080583, 0.02890014, 0.99632884],
    [-0.0964622, -0.0632074, 0.99332767]
]

Now, we will save each of these camera settings to a session file to represent our keyframes.

In [38]:
for i, position, target, up in zip(range(1, len(positions)+1), positions, targets, ups):
    ses.GetCamera().LookAt(position, target, up)
    ses.Save(f"./keyframes/keyframe{i}.vs3")

### Create helper function for linear interpolation of key frames

This function generates an animation using keyframing from a list of session paths and a corresponding list of integers that describe the number of frames between each keyframe. It creates and returns an animation that can be displayed.

In [39]:
# Function to create animation based on a number of keyframes defined in session_paths.
def animate_keyframes(session_paths, steps):
    # Load key frames as sessions
    key_frames = []
    for path in session_paths:
        ses = session.Session()
        ses.Load(path)
        key_frames.append(ses)

    # Visualization will use renderers from first session in list. Other sessions are only for camera angles
    primary_session = key_frames[0]
    anim = Animation(primary_session)
    cam = primary_session.GetCamera()
    total_frames = sum(steps)
    
    # Interpolate camera information between each key frame
    n = 0
    for i in range(len(key_frames) - 1):
        start = key_frames[i]
        end = key_frames[i+1]
        frames = steps[i]
        # Get starting information
        cam1 = start.GetCamera()
        dir1 = cam1.GetDirection()
        pos1 = cam1.GetPosition()
        up1 = cam1.GetUp()

        # Get ending information
        cam2 = end.GetCamera()
        dir2 = cam2.GetDirection()
        pos2 = cam2.GetPosition()
        up2 = cam2.GetUp()

        # Difference between camera positions on each axis
        dPositionX  = (pos2[0] - pos1[0])
        dPositionY  = (pos2[1] - pos1[1])
        dPositionZ  = (pos2[2] - pos1[2])

        # Difference between camera direction vectors on each axis
        dDirectionX = (dir2[0] - dir1[0])
        dDirectionY = (dir2[1] - dir1[1])
        dDirectionZ = (dir2[2] - dir1[2])

        # Difference between camera up vectors on each axis
        dUpX        = (up2[0] - up1[0])
        dUpY        = (up2[1] - up1[1])
        dUpZ        = (up2[2] - up1[2])

        # Linear interpolation between start and end
        for j in range(frames):
            position = [
                pos1[0]+dPositionX*j/frames,
                pos1[1]+dPositionY*j/frames,
                pos1[2]+dPositionZ*j/frames
            ]
            cam.SetPosition( position )

            direction = [
                dir1[0]+dDirectionX*j/frames,
                dir1[1]+dDirectionY*j/frames,
                dir1[2]+dDirectionZ*j/frames
            ]
            cam.SetDirection( direction )

            up = [
                up1[0]+dUpX*j/frames,
                up1[1]+dUpY*j/frames,
                up1[2]+dUpZ*j/frames
            ]
            cam.SetUp( up )
            anim.CaptureFrame()
            
            # Print statement is for each keyframe, not for whole animation
            print(f"Rendering Animation [{'#'*round((j+n)*40/total_frames)}{' '*round(40-((j+n)*40/total_frames))}] {(j+1+n)*100/total_frames:.0f}%", end="\r")
        n += steps[i]
    return anim
    

### Generate Animation

Specify the paths to your session files (keyframes) and the number of interpolation steps between each keyframe.

In [40]:
sessions = [
    "keyframes/keyframe1.vs3",
    "keyframes/keyframe2.vs3",
    "keyframes/keyframe3.vs3",
    "keyframes/keyframe4.vs3",
    "keyframes/keyframe5.vs3",
]
steps = [40,30,30,30]

### Generate and Display Animation

Create the animation using the defined keyframes and steps, then display it.

In [41]:
anim = animate_keyframes(sessions, steps)
anim.Show()

Dataset: 'wrfout_d02_2005-08-29_02.nc'
Dataset: 'wrfout_d02_2005-08-29_02.nc'
Dataset: 'wrfout_d02_2005-08-29_02.nc'
Dataset: 'wrfout_d02_2005-08-29_02.nc'
Dataset: 'wrfout_d02_2005-08-29_02.nc'


2024-06-06 16:40:52.576 python[52579:1091707] GLDRendererMetal command buffer completion error: Error Domain=MTLCommandBufferErrorDomain Code=2 "Caused GPU Hang Error (00000003:kIOGPUCommandBufferCallbackErrorHang)" UserInfo={NSLocalizedDescription=Caused GPU Hang Error (00000003:kIOGPUCommandBufferCallbackErrorHang)}
